# 如何定义图状态

本指南将涵盖定义图表状态的不同方法。

## 先决条件

- [状态概念指南](/langgraphjs/concepts/low_level/#state) - 定义图形状态的概念指南。
- [构建图表](/langgraphjs/tutorials/quickstart/) - 本指南假设您对如何构建图表有基本的了解。

## 设置

本指南需要安装 `@langchain/langgraph` 和 `@langchain/core` 软件包：
```bash
npm install @langchain/langgraph @langchain/core
```
## 入门

`Annotation` 函数是为新的 `StateGraph` 图形定义图形状态的推荐方法。`Annotation.Root` 函数用于创建顶级状态对象，其中每个字段代表图中的一个通道。

下面是如何使用一个名为 `messages` 的通道定义简单图状态的示例：

In [1]:
import { BaseMessage } from "@langchain/core/messages";
import { Annotation } from "@langchain/langgraph";

const GraphAnnotation = Annotation.Root({
  // 定义一个“消息”通道来存储 BaseMessage 对象的数组
  messages: Annotation<BaseMessage[]>({
    // 减速器功能：将当前状态与新消息结合起来
    reducer: (currentState, updateValue) => currentState.concat(updateValue),
    // 默认功能：用空数组初始化通道
    default: () => [],
  })
});

每个通道可以选择具有 `reducer` 和 `default` 功能：
- `reducer` 函数定义如何将新值与现有状态组合。
- `default` 函数为通道提供初始值。

有关Reducers的更多信息，请参阅[Reducers概念指南](/langgraphjs/concepts/low_level/#reducers)

In [2]:
const QuestionAnswerAnnotation = Annotation.Root({
  question: Annotation<string>,
  answer: Annotation<string>,
});

上面，我们所做的就是定义通道，然后将未实例化的 `Annotation` 函数作为值传递。值得注意的是，我们始终将每个通道的 TypeScript 类型作为第一个泛型参数传递给 `Annotation`。这样做可以确保我们的图状态是类型安全的，并且我们可以在定义节点时获得正确的类型。下面显示了如何从 `Annotation` 函数中提取类型：

In [3]:
type QuestionAnswerAnnotationType = typeof QuestionAnswerAnnotation.State;

这相当于以下类型：
```typescript
type QuestionAnswerAnnotationType = {
  question: string;
  answer: string;
}
```

## 合并状态

如果您有两个图形状态注释，则可以使用 `spec` 值将这两个注释合并为一个注释：

In [4]:
const MergedAnnotation = Annotation.Root({
  ...QuestionAnswerAnnotation.spec,
  ...GraphAnnotation.spec,
})

合并注释的类型是两个注释的交集：
```typescript
type MergedAnnotation = {
  messages: BaseMessage[];
  question: string;
  answer: string;
}
```
最后，使用注释实例化图形就像将注释传递给 `StateGraph` 构造函数一样简单：


In [5]:
import { StateGraph } from "@langchain/langgraph";

const workflow = new StateGraph(MergedAnnotation);

## 状态通道

`Annotation` 函数是一个方便的包装器，围绕如何在 LangGraph 中定义状态的低级实现。使用 `channels` 对象（`Annotation` 是其包装器）定义状态仍然是可能的，但在大多数情况下不建议这样做。下面的示例展示了如何使用此模式实现图表：

In [6]:
import { StateGraph } from "@langchain/langgraph";

interface WorkflowChannelsState {
  messages: BaseMessage[];
  question: string;
  answer: string;
}

const workflowWithChannels = new StateGraph<WorkflowChannelsState>({
  channels: {
    messages: {
      reducer: (currentState, updateValue) => currentState.concat(updateValue),
      default: () => [],
    },
    question: null,
    answer: null,
  }
});

上面，我们将 `question` 和 `answer` 的值设置为 `null`，因为它不包含默认值。要设置默认值，通道应按照 `messages` 键的方式实现，`default` 工厂返回默认值。`reducer` 函数是可选的，如果需要，可以将其添加到通道对象中。

## 使用 Zod

如果要向状态添加运行时验证，可以使用 Zod 而不是 `Annotation` 函数进行状态定义。

您还可以通过从 `@langchain/langgraph/zod` 导入 `registry` 来传递自定义的 `reducer` 和 `default` 工厂，这将允许您使用 LangGraph 特定方法扩展 Zod。

In [ ]:
import { registry } from "@langchain/langgraph/zod";
import { z } from "zod/v4";

const AgentState = z.object({
  query: z
    .array(z.string())
    .default(() => [])
    .register(registry, {
      reducer: {
        fn: (a, b) => a.concat(Array.isArray(b) ? b : [b]),
        schema: z.union([z.string(), z.array(z.string())]),
      },
    })
    .langgraph.reducer(
      (a, b) => a.concat(Array.isArray(b) ? b : [b]),
      z.union([z.string(), z.array(z.string())])
    ),
  question: z.string(),
  answer: z.string().min(1),
});

const graph = new StateGraph(AgentState);


对于 LangChain 消息，请使用 `z.custom<BaseMessage[]>` 架构和 `MessagesZodMeta` 以确保 LangGraph Studio 正确检测到通道。

In [ ]:
import type { BaseMessage } from "@langchain/core/messages";
import { registry } from "@langchain/langgraph/zod";
import { MessagesZodMeta } from "@langchain/langgraph";

const MessagesZodState = z.object({
  messages: z
    .custom<BaseMessage[]>()
    .default(() => [])
    .register(registry, MessagesZodMeta),
});